# Pandeia Tutorial

***

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Introduction

**Pandeia** is a high-fidelity exposure time calculator developed by STScI to characterize optimal observing setups for user-created astronomical scenes. It supports the Roman Wide Field Instrument (WFI) as well as the James Webb Space Telescope's full complement of instruments.

Due to the complexity of its simulations, Pandeia is best used on scenes that encompass ~5% of a single WFI detector. (To simulate larger areas, see the STIPS notebook tutorial.)

This notebook walks through brief examples of common use cases.

The cell below will check to ensure ancillary reference files for `pandeia` package are installed. If not, it will download the ancillary reference files and install them under your home directory (i.e., `${HOME}/refdata/`).

### Local Run Settings

If you want to run the notebook in your local machine, refer to the information in [local installation](../../markdown/local-run.md) instructions before proceeding with the notebook. The instructions provide inportant information about setting up your environment, installing dependnecies, and adding to your working directory scripts to help with the reference data installation.

Depending on which (if any) reference data are missing, this cell may take several minutes to execute.

### On the Roman Research Nexus

If you are working on the Nexus, then the ancillary reference data are pre-installed and this cell will execute instantly.

In [ ]:
import os
import sys
import importlib.util

import notebook_data_dependencies as ndd

# Download reference data (if necessary)
result = ndd.install_files(packages=['pandeia'])
ndd.setup_env(result)


## Imports

Besides the Pandeia-related imports, we will use `scipy.optimize.minimize_scalar` to help with optimizing signal-to-noise ratios (SNRs), `scipy.interpolate.interp1d` to calculate a desired target magnitude for a given observing setup and `numpy` to handle numerical computing.

In [ ]:
from pandeia.engine.calc_utils import build_default_calc
from pandeia.engine.perform_calculation import perform_calculation
from scipy.optimize import minimize_scalar
from scipy import interpolate
import numpy as np

We set `FILTER` as global variable before beginning since all examples make use of the same F129 imaging filter. Please note that Pandeia's filter definition is case-sensitive and will only take lower-case letters for filter names. 

In [ ]:
FILTER = 'f129'     # Pandeia's filter definitaion is case-sensitive and will only take lower-case letters for filter names 

***

## Examples

### Calculate a Scene's Signal-to-Noise Ratio

In this first example, we calculate the expected SNR for a point source with a flat spectral distribution (default target) normalized to 25 AB magnitudes. We place the source on Detector #1 (SCA01) and take three exposures in band F129 with the multi-accumulation (MA) table \"c2a_img_hlwas\", truncated after 9 resultants (407.96 seconds of total exposure time). MA tables describe the sequence of individual reads that are combined into resultants and comprise the up-the-ramp sampling during a single exposure of the WFI detectors. For more information on the WFI detectors, please refer to the RDox documentation on [WFI](https://roman-docs.stsci.edu/roman-instruments-home/wfi-imaging-mode-user-guide/wfi-design/description-of-wfi) and for the MA tables, please refer to the RDox documentation on [MA tables](https://roman-docs.stsci.edu/raug/astronomers-proposal-tool-apt/appendix/appendix-wfi-multiaccum-tables).

We first create a default calculation using Pandeia's built-in function `build_default_calc(<telescope>, <instrument>, <mode>)`: 

In [ ]:
calc = build_default_calc('roman', 'wfi', 'imaging')        # Creating a default calculation using Roman's WFI with an imaing mode

Let's take a look at how the default calculation is set up:

In [ ]:
print(calc)                                                 # Print the default calculation

The `build_default_calc` created a scene with a single point source placed on Detector 01 (SCA01), to be observed in a single exposure using the F158 filter and the \"c2a_img_hlwas\" MA table, with no truncation. Note that for the Roman WFI, the term "exposure" refers to a multi-accum sequence of the detector array at a single dither point in the dither pattern. Next, we define the observing setup and make some changes to the default settings:

In [ ]:
calc['configuration']['instrument']['filter'] = FILTER      # Setting the filter to F129
calc['configuration']['detector']['nexp'] = 3               # Taking three exposures of multi-accum sequence
calc['configuration']['detector']['nresultants'] = 9        # Truncate after 9 resultant

Next, we normalize the default point source flux to 25 AB magnitudes:

In [ ]:
mag = 25
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'

Finally, we perform the signal to noise calculation using Pandeia's built-in function `perform_calculation` and print the results:

In [ ]:
report = perform_calculation(calc)
sn = report['scalar']['sn']
print(f'Estimated S/N: {sn:.2f}')

Note that this step may generate a WARNING from synphot that the spectrum is extrapolated, which can be safely ignored.

Running Pandeia for Roman may return a warning such as: `if np.log(abs(val)) < -1*precision and val != 0.0`. This is related to a JWST-specific test for float precision, and can be ignored.

### Calculating Limiting Magnitudes and Optimizing the Number of Exposures for Roman WFI Simulations

In the next example, we start by determining the magnitude corresponding to a given signal-to-noise ratio (SNR) for a specific setup. We then extend this analysis to calculate the optimal number of exposures needed to achieve a target SNR for a given source flux.

The following helper functions use Pandeia to simulate a range of scenes at different magnitudes in order to estimate the magnitude corresponding to a given SNR for a specific number of exposures. As above, we assume a point source with a flat spectrum, and the MA table is set to the "c2a_img_hlwas" table without any truncation.

#### Step 1: Calculating Limiting Magnitude for a Given Setup

In the first step, we estimate the limiting magnitude for a point source at a desired SNR. This process involves iterating over a range of magnitudes, calculating the SNR for each, and interpolating the results to determine the magnitude that corresponds to the target SNR. The observing parameters include the number of exposures and the specified filter.

Example Use Case:

SNR = <span style="color:red">5</p>

Number of exposures = <span style="color:red">10</p>

Filter = <span style="color:red">F129</p>

In [ ]:
def compute_mag(filt, nexp, bracket=(18, 30)):
    """
    Method to compute the magnitude from S/N and number of exposures
 
    Parameters
    ----------
    filt : str
        Name of Roman WFI filter
    nexp : int
        Number of exposures
    bracket : tuple
        Range of magnitudes to test. default: (18, 30)

 
    Returns
    -------
    mag_range : float
        An array of magnitudes used to compute the SNRs
    computed_snrs: float
        An array of computed SNRs from Pandeia calculations
    """
 
    # Set up default Roman observation
    calc = build_default_calc('roman', 'wfi', 'imaging')
 
    # Modify defaults to place a source with an AB magnitude
    calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
    calc['scene'][0]['spectrum']['normalization']['norm_waveunit'] = 'um'
      
    # Set number of exposures and filter
    calc['configuration']['detector']['nexp'] = nexp
    calc['configuration']['instrument']['filter'] = filt

    # Create an array of magnitudes range of interest
    mag_range = np.arange(bracket[0], bracket[1]+1, 1)
    # Create empty lists to save the computations
    computed_snrs = []
    # Compute the SNRs for a given magnitude
    for m in range(len(mag_range)):
        mag = mag_range[m]
        calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
        report = perform_calculation(calc)
        computed_snrs.append(report['scalar']['sn'])

    return mag_range, computed_snrs

def _mag2sn_(mag_range, computed_snrs, sntarget):
    """
    Calculate a magnitude given a desired SNR by interpolating (computed_snrs, mag_range) from compute_mag
    
    Parameters
    ----------
    mag_range: float
        An array of magnitudes used in calculating a range of SNRs in compute_mag
    computed_snrs: float
        An array of computed SNR given the mag_range using Pandeia calculation object
    sntarget: float
        Required S/N
    
    """
    interpolator = interpolate.interp1d(computed_snrs, mag_range)
    mag = interpolator(sntarget)

    return mag

In [ ]:
sn = 5
nexp = 10

# mag, report = compute_mag(FILTER, sn, nexp)
mag_range, computed_snrs = compute_mag(FILTER, nexp)
mag = _mag2sn_(mag_range, computed_snrs, sn)
print(f'Estimated magnitude: {mag:.2f}')

#### Step 2: Determining Optimal Number of Exposures
Once the magnitude is determined, we calculate the optimal number of exposures needed to achieve a target SNR for a known flux. This involves simulating observations with varying exposure counts and identifying the minimum number required to meet or exceed the target SNR. This approach ensures efficient use of telescope time while maintaining data quality.

The following helper functions use Pandeia to simulate a range of scenes with different numbers of exposures in order to estimate the optimal observing time to reach the expected limiting magnitude for a source with a given flux. As above, we assume a point source with a flat spetrum, and the MA table is set to the "c2a_img_hlwas" table, truncated to 8 resultants.

Example Use Case:

SNR = <span style="color:red">20</span>

Magnitude = <span style="color:red">26.84</span> (calculated from above, or known)

Filter = <span style="color:red">F129</span>

In [ ]:
def _nexp2sn_(nexp, calc, sntarget):
    """
    Optimize a S/N ratio given a number of exposures. This is a helper function
    used as an argument for scipy's minimize_scalar() as used in compute_mag().
    
    Parameters
    ----------
    nexp : int
        The number of exposures used in an iteration of minimize_scalar()
    calc : 
        A Pandeia calculation object
    sntarget : 
        Required S/N from the matching argument of compute_mag()
    """
    calc['configuration']['detector']['nexp'] = int(nexp)
    etc = perform_calculation(calc)['scalar']
    
    return (sntarget - etc['sn'])**2


def compute_nexp(filt, sn, mag, bracket=(1, 1000), xtol=0.1):
    """
    Method to compute the number of exposures from S/N and magnitude
 
    Parameters
    ----------
    filt : str
        Name of Roman WFI filter
    sn : float
        Required S/N
    mag : float
        AB Magnitude of source
    bracket : tuple, default (1, 1000)
        Range of magnitudes to test
    xtold: float, default 0.1
        Target tolerance for minimizer
 
    Returns
    -------
    nexp : float
        Optimal number of exposures for specified S/N and magnitude
    report: dict
        Pandeia dictionary with optimal parameters
    exptime: float
        Exposure time for optimal observation
    """
 
    # Set up default Roman observation
    calc = build_default_calc('roman', 'wfi', 'imaging')
 
    # Modify defaults to place a source with an AB magnitude
    calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
    calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
    calc['scene'][0]['spectrum']['normalization']['norm_waveunit'] = 'um'
      
    # Set filter
    calc['configuration']['instrument']['filter'] = filt
 
    # Check that the minimum of 1 exposure has a S/N lower than requested,
    # otherwise there is no sense in attempting to minimize nexp.
    calc['configuration']['detector']['nexp'] = 1
    calc['configuration']['detector']['nresultants'] = 8
    report = perform_calculation(calc)
    
    if report['scalar']['sn'] > sn:
        nexp = 1
    else:
        res = minimize_scalar(_nexp2sn_, bracket=bracket, bounds=bracket,
                              args=(calc, sn), method='bounded',
                              options={'xatol': xtol})
        
        # Take the optimization result and set it to nexp
        # 'x' is the solution array in the optimization result object
        # For more details on the minimize_scalar function, refer to https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html
        nexp = int(res['x'])       
        calc['configuration']['detector']['nexp'] = nexp
        report = perform_calculation(calc)
 
        # this generally returns a S/N less than the required amount.
        # let's ensure that we get *AT LEAST* the required S/N for 2 reasons:
        # 1) better to err on the side of caution
        # 2) make code consistent with the above if clause
        if report['scalar']['sn'] < sn:
            nexp += 1
             
    exptime = report['scalar']['total_exposure_time']
         
    return nexp, report, exptime

For example, we can use the functions above to determine the optimal number of exposures to reach a SNR of 20 when observing a point source of magnitude 26.84 in the F129 band:

In [ ]:
sn = 20.
mag = 26.84
 
nexp, etc, exptime = compute_nexp(FILTER, sn, mag)
print(f'number of exposures: {nexp}')
print(f'actual S/N reached: {etc["scalar"]["sn"]:.2f}')
print(f'Exposure time: {exptime:.2f}')

### Modifying the Spectral Energy Distribution

While previous examples assume a point source with a flat SED, Pandeia also offers the ability to use a variety of different shapes and spectral inputs. In the example below, we calculate the SNR for an A0V star (Phoenix model) of magnitude 25 AB, observed in the F129 band, with 3 exposures of the default MA table "c2a_img_hlwas", with no truncation. For more information on how to implement complex scenes with a variety of shapes and SEDs, please refer to the [JWST Tutorials](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-quickstart#PandeiaQuickstart-Samplecode).

In [ ]:
calc = build_default_calc('roman', 'wfi', 'imaging')

nexp = 3
calc['configuration']['detector']['nexp'] = nexp
calc['configuration']['instrument']['filter'] = FILTER

In [ ]:
mag = 25
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'
 
calc['scene'][0]['spectrum']['sed']['sed_type'] = 'phoenix'
calc['scene'][0]['spectrum']['sed']['key'] = 'a0v'

In [ ]:
report = perform_calculation(calc)
sn = report['scalar']['sn']
print(f'Estimated S/N: {sn:.2f}')

### Observing NGC2506-G31 with Roman WFI

In this example, we show a real science case using NGC2506-G31, a G1V standard star that is used as a cross-mission calibration standard for both JWST and HST observations. We would like to see if Roman can observe the same star and if so, with which observing setup. We are interested in placing the star on Detector 11 (SCA11). 

In [ ]:
calc = build_default_calc('roman', 'wfi', 'imaging')        # Creating a default calculation using Roman's WFI with an imaing mode

calc['configuration']['instrument']['filter'] = FILTER      # Setting the filter to F129
calc['configuration']['instrument']['detector'] = 'sca11'   # Setting the detector fo SCA11
calc['configuration']['detector']['nexp'] = 1               # Taking one exposure of multi-accum sequence

calc['configuration']['detector']['ma_table_name'] = 'c2a_img_hlwas'    # Using the default MA table
calc['configuration']['detector']['nresultants'] = -1       # No truncation of the MA table


# Setting up the source SED
calc['scene'][0]['spectrum']['sed']['sed_type'] = 'phoenix'
calc['scene'][0]['spectrum']['sed']['key'] = 'g2v'          # Using the closest spectral type available to G1V


# Setting up the normalization parameters
calc['scene'][0]['spectrum']['normalization']['type'] = 'photsys'
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = 16.260     # K-band magnitude of the source
calc['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'vegamag'
calc['scene'][0]['spectrum']['normalization']['bandpass'] = '2mass,ks'


# Run the calculation
report = perform_calculation(calc)

After running the calculation, we can check for any warning messages in the report. These warnings can be accessed through the "warnings" dictionary.

In [ ]:
print(report['warnings'])

We observe that a pixel is partially saturated. If you are concerned about a partially saturated pixel, you can check the report determine the maximum number of resultants needed to avoid saturation of the brightest pixel on the detector. This information can be accessed through the "sat_nresultants" key within the "scalar" dictionary.

In [ ]:
print(report['scalar']['sat_nresultants'])

Let's manually verify that the value is indeed nresultant = 5. The maximum nresultant for the "c2a_img_hlwas" MA table is 10, and we will calculate backward, starting from no truncation and incrementally applying truncation. 

In [ ]:
for n in reversed(range(10)):
    # Check to see if there is any warning message about the saturation (both partial and full)
    # Stop the loop if the saturation warning no longer exists
    # Adding 1 for the resultant because python indexing starts with 0. 
    resultant = n + 1
    calc['configuration']['detector']['nresultants'] = resultant
    report = perform_calculation(calc)
    partial = report['warnings'].get('partial_saturated')
    full = report['warnings'].get('full_saturated')

    if (partial == None) and (full == None):
        nresultant = resultant
        print(f'No saturation happens with resultant {nresultant}')
        break

The answer turns out to be 5 indeed.

Note that the Pandeia engine does not support simulations at specific coordinates or times. Instead, the engine includes seven predefined backgrounds at two different locations. For more details, refer ot the JDox documentation on [Pandeia backgrounds](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-backgrounds#gsc.tab=0). If you want to see how the SNR changes over time at the specific location of this star, you will need to use the web application of the ETC, available at the [Roman WFI ETC](roman.etc.stsci.edu).

## Additional Resources

- The Roman User Documentation's ["Pandeia for Roman"](https://roman-docs.stsci.edu/simulation-tools-handbook-home/pandeia-for-roman) page and associated overview.
- Full API references for [Pandeia Engine inputs](https://outerspace.stsci.edu/display/PEN/Pandeia+Engine+Input+API) and [Pandeia Engine outputs](https://outerspace.stsci.edu/display/PEN/Pandeia+Engine+Output+API).
- The [Roman Help Desk](https://roman-docs.stsci.edu/roman-help-desk-at-stsci), an official outlet for user questions about Pandeia.
- [Pandeia JWST Tutorials](https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-quickstart#PandeiaQuickstart-Samplecode)

## About this Notebook

**Author:** Justin Otor, Eunkyu Han, Harish Khandrika      
**Updated On:** 2025-01-09

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 